In [15]:
import tqdm
import re
import numpy as np
from collections import Counter
from itertools import product
import random

In [16]:
class LanguageModel():
    def __init__(self, order=2):
        self.order=order
        
    def product(self, nums):
        "Multiply the numbers together.  (Like `sum`, but with multiplication.)"
        result = 1
        for x in nums: result *= x
        return result

    def get_ngrams(self, tokens, n):
        return [' '.join(tokens[i:i+n]) for i, token in enumerate(tokens)]
    
    def get_counts(self, corpus, order):  
        counts = {'n' + str(i) : Counter(self.get_ngrams(corpus, n=i)) for i in range(1, order+1)}
        counts['n0'] = {'':len(corpus)}
        return counts
    
    def get_prob(self, counts, word, context=''):
        '''With Laplace shoothing as yet.
        Not for public use.'''
        order = len(context.split())+1
        separator = ' ' if order > 1 else ''
        return (counts['n'+str(order)][separator.join([context, word])] + 1) / \
               (counts['n'+str(order-1)][context] + len(counts['n'+str(order)]))
        
    def get_logprob(self, counts, word, context=''):
        return np.log(self.get_prob(counts, word, context))
    
    def get_following(self, counts, context):
        '''Slow as hell. 
        To optimize might use embedded dictionaries.'''
        order = len(context.split())+1
        return sorted(
            [(k.split()[-1], v, self.get_prob(counts, k.split()[-1], context)) \
            for k, v in counts['n'+str(order)].items()                         \
            if re.match(context+' '+'\w+', k)],                                \
            key=lambda x:x[1], reverse=True)   
    
    def get_string_probs(self, counts, string, order, log=True):
        prob_fun = self.get_logprob if log else self.get_prob
        tokens = string.split()
        probs = []
        for i in range(len(tokens)):
            context = ' '.join(tokens[i-order+1:i]) if i>=order else ' '.join(tokens[:i])
            prob = prob_fun(counts, word = tokens[i], context = context)
            probs.append(prob)
        return probs
    
    def interpolate(self, counts, string, order, log=True, lambdas='default'):
        lmbd = [0.3, 0.7, 0.0] if lambdas == 'default' else lambdas
        aggregate = sum if log else self.product
        probs = [self.get_string_probs(counts, string, order=i, log=log) \
                 for i in range(1, order+1)]
        probs_interpolated = []
        for tup in zip(*probs):
            prob_token = 0
            for i in range(len(tup)):
                prob_token += tup[i] * lmbd[i]
            probs_interpolated.append(prob_token)
        return aggregate(probs_interpolated)
    
    def fit(self, corpus):
        self.counts = self.get_counts(corpus, self.order)
        
    def prob(self, string, log=False):
        return self.interpolate(self.counts, string, self.order, log=log)
    
    def context_prob(self, word, context='', log=False):
        prob_fun = self.get_logprob if log else self.get_prob
        c = context.split()
        history = ' '.join(c) if len(c) < self.order else ' '.join(c[-self.order+1:])
        return prob_fun(self.counts, word, history)  
    
    def following(self, context):
        c = context.split()
        history = ' '.join(c) if len(c) < self.order else ' '.join(c[-self.order+1:])
        return self.get_following(self.counts, history)

In [213]:
class Candidator():
    def __init__(self, dictionary, abc='йцукенгшщзхъфывапролджэячсмитьбю', check_all=True):
        self.abc = abc
        self.dictionary = set(dictionary)
        self.check_all = check_all
        
    def edits(self, word): # mostly from norvig, modified for faster dict search
        letters = self.abc
        d = self.dictionary
        splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
        deletes = [ed for ed in [L + R[1:] for L, R in splits if R] if ed in d]
        transposes = [ed for ed in [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1] \
                      if ed in d]
        replaces = [ed for ed in [L + c + R[1:] for L, R in splits if R for c in letters] \
                    if ed in d]
        inserts = [ed for ed in [L + c + R for L, R in splits for c in letters] if ed in d]
        return set(deletes + transposes + replaces + inserts + [word])
    
    def get_sentences(self, text):
        return text.lower().split('. ')
    
    def word_candidates(self, sent):
        if self.check_all:
            return [self.edits(word) for word in sent.lower().split()]
        else:
            return [self.edits(word) if (word not in self.dictionary) else {word} for word in sent.lower().split()]
    
    def sent_candidates(self, sent):
        return [candidate for candidate in product(*self.word_candidates(sent))]
    
    def candidates(self, sent):
        return [' '.join(candidate) for candidate in self.sent_candidates(sent)]

In [214]:
class Ranker():
    def __init__(self, lang_model, candidator):
        self.lm = lang_model
        self.c = candidator

In [215]:
def cleanse(s, rgxp = '[\W\da-z]'):
    return re.sub(' +', ' ', re.sub(rgxp, ' ', s.lower()))

In [216]:
%%time
with open('corpora.txt', encoding='utf-8') as f:
    tokens1 = cleanse(f.read().lower()).split()

Wall time: 4.91 s


In [217]:
%%time
f = open('hagen-orf.txt', encoding='utf-8').readlines()
d2 = [w for w in [re.findall('^ *(.+) |', l)[0].split(' | ')[0] for l in f] if len(w)>0]
len(d2)

Wall time: 6.48 s


In [218]:
%%time
c = Candidator(d2, check_all=True)
lm = LanguageModel(order=3)
lm.fit(tokens1)

Wall time: 7.46 s


In [219]:
lm.prob('есть', log=True)

-6.6856971271622356

In [220]:
%%time
cand_list = c.candidates('школа жлословия')
cand_probs = sorted([(cand, lm.prob(cand, log=True)) for cand in cand_list], 
                    key=lambda x:x[1], reverse=True)

Wall time: 0 ns


In [221]:
cand_probs[:10]

[('школа злословия', -21.681645158124368),
 ('школы злословия', -22.008580075754729),
 ('школе злословия', -22.404439278478257),
 ('школу злословия', -22.83886547498442),
 ('школы жлословия', -23.263267909385149),
 ('школ злословия', -23.592321050483058),
 ('школе жлословия', -23.659127112108685),
 ('школах злословия', -23.782358219701798),
 ('школа жлословия', -23.813267069701549),
 ('школу жлословия', -24.09355330861484)]

In [224]:
%%time
c = Candidator(d2, check_all=False)

Wall time: 165 ms


In [225]:
%%time
cand_list = c.candidates('пошел на жработу')
cand_probs = sorted([(cand, lm.prob(cand, log=True)) for cand in cand_list], 
                    key=lambda x:x[1], reverse=True)

Wall time: 5.24 ms


In [226]:
cand_probs[:10]

[('пошел на работу', -29.224724933416912),
 ('пошел за работу', -32.569159002246984),
 ('пошел на жработу', -33.906827092220198),
 ('пошел наш работу', -35.070000246233178),
 ('пошел та работу', -35.151055180751129),
 ('пошел га работу', -35.633803634979557),
 ('пошел сна работу', -35.666561277968881),
 ('пошел за жработу', -35.763585406652481),
 ('пошел дна работу', -35.831567315454464),
 ('пошел па работу', -35.874496138793504)]

In [234]:
c.word_candidates('россия ржссия федерация фодерация')

[{'россия'}, {'ржссия', 'россия'}, {'федерация'}, {'федерация', 'фодерация'}]

In [239]:
%%time
cand_list = c.candidates('в ржссию')
cand_probs = sorted([(cand, lm.prob(cand, log=True)) for cand in cand_list], 
                    key=lambda x:x[1], reverse=True)

Wall time: 0 ns


In [240]:
cand_probs[:10]

[('в россию', -12.528359433602237),
 ('в ржссию', -17.351424104611478),
 ('я россию', -18.277298741027003),
 ('я ржссию', -19.869779050247363),
 ('ив россию', -24.641096271347109),
 ('ив ржссию', -26.233576580567469)]